In [1]:

using Revise
includet("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [2]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
fermion=false
α= α3b(fermion,0.0,0.0,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.0 T=0.0 parity=1 Number of channels: 1
scaling factor for x: 0.3


Main.mesh.meshset(12, 12, 12, [-0.9815606342467192, -0.9041172563704748, -0.7699026741943047, -0.5873179542866175, -0.3678314989981802, -0.1252334085114689, 0.1252334085114689, 0.3678314989981802, 0.5873179542866175, 0.7699026741943047, 0.9041172563704748, 0.9815606342467192], [0.04717533638651175, 0.10693932599531826, 0.16007832854334625, 0.20316742672306587, 0.23349253653835492, 0.24914704581340288, 0.24914704581340288, 0.23349253653835492, 0.20316742672306587, 0.16007832854334625, 0.10693932599531826, 0.04717533638651175], [0.0848575044719742, 0.2856978124093846, 0.6049476415733046, 1.0477062209334076, 1.6216473060047178, 2.337844182123722, 3.2122166066967295, 4.268145709136995, 5.5415989902690015, 7.092535125720268, 9.036017587831902, 11.666785312828601], [0.14277897009800458, 0.259383321803658, 0.37995967777078926, 0.5068231073270444, 0.6428816610948332, 0.7921245262743847, 0.960438937417142, 1.1572465340299454, 1.3992307171257872, 1.7204236864089044, 2.2079451522012565, 3.2157606

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [ ]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [ ]:
Gαα=computeGcoefficient(α, grid)

In [ ]:
using Revise
includet("twobody.jl")
using .twobodybound

In [ ]:
potname="MT"
bound_states=bound2b(grid, potname)

In [ ]:
using Revise
includet("spline.jl")
using .spline

In [ ]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

In [ ]:
using Printf 
test_cubherm_interpolation()

In [3]:
using Revise
includet("matrices.jl")
using .matrices


In [12]:
Rxy=Rxy_matrix(α, grid)
println("Rxy matrix:", Rxy[30,144])

Rxy matrix:1.5928843070523994e-5 + 0.0im


In [5]:
using Revise
includet("threebodybound.jl")
using .threebodybound

In [14]:
ThreeBody_Bound(α, grid)

H: -0.00013180026243237458 + 0.0im
Eigenvalues: ComplexF64[-3.9345407925632827 + 1.304512053934559e-15im, -3.368713034086194 - 4.7467108565780734e-17im, 0.23143388615811028 - 5.249034195620628e-16im, 0.5920316796433372 - 2.183879699594378e-17im, 0.6367965929160415 - 2.8105535408502634e-16im, 0.6663033852741104 + 0.0005567667066880285im, 0.6663033852741107 - 0.0005567667066879923im, 0.6765284133707582 - 0.03119111244116814im, 0.6765284133707588 + 0.03119111244116806im, 0.7988600119774909 - 1.3025091930026865e-16im, 0.8309539320547615 - 0.20301954905897274im, 0.8309539320547632 + 0.20301954905897326im, 2.8922534602596928 - 0.19001766690790492im, 2.8922534602596928 + 0.19001766690790467im, 4.023403435336699 - 8.834949028407007e-16im, 4.398791489056579 - 1.3005849748635433e-15im, 4.473544594022157 - 3.4617580027130726e-16im, 4.522630903381054 + 2.321922429926712e-15im, 4.528506676533756 + 1.6102019570549616e-17im, 4.56491433069822 - 0.027252542542322494im, 4.564914330698234 + 0.02725254254

2-element Vector{Any}:
 (-3.9345407925632827 + 1.304512053934559e-15im, ComplexF64[3.331573720495454e-5 + 8.147398161402301e-20im, -0.0007238257090607665 - 3.644782772035254e-18im, -0.0031713770681560394 + 3.9302328752599536e-19im, -0.002767679172429296 - 2.913793338554793e-19im, 0.00684513762843334 - 1.0570971181733668e-18im, -0.00047237971700020556 - 2.0125502826762176e-18im, 0.003406430338432059 + 8.402566836762659e-19im, 0.0001895232217715061 + 1.7580168820263004e-18im, -0.0031978993049309483 + 2.2158381900172497e-18im, 0.0023744484085609337 + 6.0308745844506184e-18im  …  0.015693090957288224 - 1.2685165418080402e-17im, 0.053843521703726936 + 3.133344278483108e-17im, 0.01639794537716859 + 1.8458541986565713e-17im, 0.01997037190775935 + 7.968885967768458e-18im, -0.010097300505459306 + 1.021860547567588e-17im, -0.011078096658703043 + 1.3511869574600599e-17im, 0.020789821233123493 + 9.893344823930228e-18im, -0.011052193035199356 - 3.6700243538634325e-17im, 0.0025856668859775473 - 1.86

In [ ]:
T_matrix(α,grid) 

In [ ]:
V_matrix(α, grid, "MT")